In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import torchvision.utils as vutils
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import image_loader

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5))
    ])

In [8]:
n_noise = 100
n_channel = 3
batch_size = 100

In [9]:
train_X = image_loader.load('../../datasets/celebA/img_align_celeba/cropped/')
train_Y = [1] * len(train_X)

train_X = torch.tensor(train_X)
train_Y = torch.tensor(train_Y)

In [10]:
train = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train, batch_size=100, shuffle=True, num_workers=2)

In [11]:
len(train_X)

4000

In [12]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1, 1, (batch_size, n_noise))

In [13]:
class Generator(nn.Module):
    
    def __init__(self):
        super(Generator, self).__init__()
        
        # 1, 100
        self.fc1 = nn.Linear(n_noise, 256*7*7)
            
        self.conv = nn.Sequential(
            
            nn.ConvTranspose2d(256, 128, 3, 2, 1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.ConvTranspose2d(128, 64, 3, 2, 1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.ConvTranspose2d(64, 32, 3, 2, 1, output_padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.ConvTranspose2d(32, 16, 3, 2, 1, output_padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            nn.ConvTranspose2d(16, 8, 3, 2, 1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
        
            nn.ConvTranspose2d(8, n_channel, 3, 1, 1)
        )
        
    def forward(self, x):
        x = self.fc1(x)
        x = x.reshape(-1, 256, 7, 7)
        x = self.conv(x)
        return F.tanh(x)

In [14]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(n_channel, 8, 3, 1, 1),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(8, 16, 3, 1, 1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(16, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2,2),
            

            nn.Conv2d(256, 256, 3, 1, 1)       
        )
        
        self.fc1 = nn.Linear(256*7*7, 1)
        
    def forward(self, x):
        x = self.conv(x).reshape(-1, 256*7*7)
        x = self.fc1(x)
        
        return F.sigmoid(x)


In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

G = Generator()
D = Discriminator()

G = G.to(device)
D = D.to(device)

criterion = nn.BCELoss()
optimizerG = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerD = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [17]:
print("Training start")
start = time.time()

for epoch in range(500):
    for data, target in train_loader:
        G.train()
        train_x, _ = data.reshape(-1, 3, 224, 224).to(device), target
        
        #training Discriminator
        optimizerD.zero_grad()
        d_real_data = train_x
        d_real_output = D(d_real_data)
        d_real_error = criterion(d_real_output, torch.ones(batch_size).reshape(batch_size,1).to(device))
        
        d_gen_input = torch.from_numpy(get_noise(batch_size, n_noise)).float().to(device)
        d_fake_data = G(d_gen_input.float()).detach()
        d_fake_output = D(d_fake_data)
        d_fake_error = criterion(d_fake_output, torch.zeros(batch_size).reshape(batch_size,1).to(device))

        d_train_loss = d_real_error + d_fake_error
        d_train_loss.backward()
        optimizerD.step()
        
        
        #training Generator
        optimizerG.zero_grad()
#         g_data = torch.from_numpy(get_noise(batch_size, n_noise)).float().to(device)
        g_fake_output = G(d_gen_input)
        gd_fake_output = D(g_fake_output)
        g_error = criterion(gd_fake_output, torch.ones(batch_size).reshape(batch_size,1).to(device))
        g_error.backward()
        
        optimizerG.step()
    
    
        lossD = d_train_loss
        lossG = g_error
        
    G.eval()
    print('epoch %d'%epoch, 'lossD',lossD.data,'lossG',lossG.data)
    image = G(torch.from_numpy(get_noise(10, n_noise)).to(device).float())
    fig, ax = plt.subplots(1, 10, figsize=(15,4))
    
    for i in range(10):
        data = image[i].cpu().data.numpy().reshape(3, 224, 224)
        data = np.transpose(data, (1,2,0))
        ax[i].set_axis_off()
        ax[i].imshow((data+1)/2)
    
    plt.savefig('./DCGAN_pytorch_celebA_samples/{}.png'.format(str(epoch+200).zfill(3)), bbox_inches='tight')
    plt.close()
        
end = time.time()

print('Elapsed Time : %.3f sec'%(end-start))

Training start
epoch 0 lossD tensor(0.1086, device='cuda:0') lossG tensor(4.6436, device='cuda:0')
epoch 1 lossD tensor(0.1109, device='cuda:0') lossG tensor(6.0224, device='cuda:0')
epoch 2 lossD tensor(0.0577, device='cuda:0') lossG tensor(6.0779, device='cuda:0')
epoch 3 lossD tensor(0.0244, device='cuda:0') lossG tensor(7.6587, device='cuda:0')
epoch 4 lossD tensor(0.1948, device='cuda:0') lossG tensor(11.0750, device='cuda:0')
epoch 5 lossD tensor(0.0514, device='cuda:0') lossG tensor(6.4536, device='cuda:0')
epoch 6 lossD tensor(0.1034, device='cuda:0') lossG tensor(6.3645, device='cuda:0')
epoch 7 lossD tensor(0.0510, device='cuda:0') lossG tensor(7.4124, device='cuda:0')
epoch 8 lossD tensor(0.0797, device='cuda:0') lossG tensor(6.3077, device='cuda:0')
epoch 9 lossD tensor(0.1140, device='cuda:0') lossG tensor(8.7058, device='cuda:0')
epoch 10 lossD tensor(0.0240, device='cuda:0') lossG tensor(7.0328, device='cuda:0')
epoch 11 lossD tensor(0.1099, device='cuda:0') lossG tensor

In [18]:
torch.save(G, 'dcgan_celebA_G.pt')
torch.save(D, 'dcgan_celebA_D.pt')

C:\Users\my\Anaconda3\envs\pytorch\lib\site-packages\torch\serialization.py:250: UserWarning: Couldn't retrieve source code for container of type Generator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\my\Anaconda3\envs\pytorch\lib\site-packages\torch\serialization.py:250: UserWarning: Couldn't retrieve source code for container of type Discriminator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
G.eval()
n = get_noise(1,100)
image = G(torch.from_numpy(n).cuda().float())

plt.imshow(image.data.cpu().numpy().reshape(224,224))

In [ ]:
image = G(torch.from_numpy(get_noise(10, n_noise)).to(device).float())
fig, ax = plt.subplots(1, 10, figsize=(15,4))
for i in range(10):
    data = image[i].cpu().data.numpy().reshape(3, 224, 224)
    data = np.transpose(data, (1,2,0))
    ax[i].set_axis_off()
    ax[i].imshow(data)
